In [4]:
 %pip install -q "spacy>3.0.0" pandas sklearn

Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [8 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Emily\AppData\Local\Temp\pip-install-vnx6iv74\sklearn_d6bcb7054403473582f20d9e21e83ac6\setup.py", line 10, in <module>
      LONG_DESCRIPTION = f.read()
    File "C:\Users\Emily\anaconda3\lib\encodings\cp1252.py", line 23, in decode
      return codecs.charmap_decode(input,self.errors,decoding_table)[0]
  UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 7: character maps to <undefined>
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above

In [39]:
!python -m spacy download en_core_web_lg

     -------------------------------------- 400.7/400.7 MB 7.2 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.3.0/en_core_web_lg-3.3.0-py3-none-any.whl#egg=en_core_web_lg==3.3.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617


In [46]:
import pandas as pd

pd.options.display.max_colwidth = None
pd.options.display.max_rows = 6
data = pd.read_csv('Symptom2Disease.csv')
#data = data.sample(frac=1).reset_index(drop=True)
data

,id,diganosis,diganosis.1,text
0,0,PSORIASIS,Psoriasis,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,1,PSORIASIS,Psoriasis,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,2,PSORIASIS,Psoriasis,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
...,...,...,...,...
1197,297,DIABETES,diabetes,I regularly experience these intense urges and the want to urinate. I frequently feel drowsy and lost. I've also significantly lost my vision.
1198,298,DIABETES,diabetes,"I have trouble breathing, especially outside. I start to feel hot and start to sweat. I frequently have urinary tract infections and yeast infections."
1199,299,DIABETES,diabetes,"I constantly sneeze and have a dry cough. My infections don't seem to be healing, and I have palpitations. My throat does ache occasionally, but it usually gets better."


In [47]:
cats = data.diganosis.unique().tolist()
cats

['PSORIASIS',
 'VARICOSE VEINS',
 'TYPHOID',
 'CHICKEN POX',
 'IMPETIGO',
 'DENGUE',
 'FUNGAL INFECTION',
 'COMMON COLD',
 'PNEUMONIA',
 'DIMORPHIC HEMORRHOIDS',
 'ARTHRITIS',
 'ACNE',
 'BRONCHIAL ASTHMA',
 'HYPERTENSION',
 'MIGRAINE',
 'CERVICAL SPONDYLOSIS',
 'JAUNDICE',
 'MALARIA',
 'URINARY TRACT INFECTION',
 'ALLERGY',
 'GASTROESOPHAGEAL REFLUX DISEASE',
 'DRUG REACTION',
 'PEPTIC ULCER DISEASE',
 'DIABETES']

In [48]:
from typing import Set, List, Tuple 
from spacy.tokens import DocBin
import spacy

#load spacy pretrained model that we downloaded before 
nlp = spacy.load("en_core_web_md")

# Create function to ccreate spacy dataset
def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    docs = DocBin()
    # use nlp.pipe to efficiently process a large number of text inputs,
    # the as_tuple arguments enables giving a list of tuples as input 
    # and reuse it in the loop, here for the labels 
    
    for doc, label in nlp.pipe(data, as_tuples=True):
        # Encode the labels (assign 1 to diganosis)
        for cat in cats: 
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
        docs.to_disk(target_file)
        return docs 

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["text"].values, data["diganosis"].values, test_size=0.2, random_state=1)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.1, random_state=1)
make_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
make_docs(list(zip(X_dev, y_dev)), "dev.spacy", cats=cats)
make_docs(list(zip(X_test, y_test)), "test.spacy", cats=cats)

In [15]:
train_data = data[:900]
dev_data = data[900:1000]
test_data = data[1000:]

print(train_data)

make_docs(train_data, "train.spacy", cats=cats)
make_docs(dev_data, "dev.spacy", cats=cats)
make_docs(test_data, "test.spacy", cats=cats)

[("I have seen rashes on my arms and neck and it itches if I scratch them. I've also had a high fever for a few days. I have no idea what is causing it. The itching is causing me a lot of discomforts.", 'CHICKEN POX'), ("I've been having this really annoying itch all over my body, and I have red and bumpy spots on my skin too. There are also some areas that are a different color than the rest of my skin, and I've had some bumps that look like little balls.", 'FUNGAL INFECTION'), ('My nails are slightly dented. Even my joints are now experiencing excruciating discomfort. My skin has a silver-like powder, especially on my back and elbows. ', 'PSORIASIS'), ('The joint pain I have been experiencing is severe and feels like a constant ache. My head aches most of the time and I am starting to develope mild fever, accompanied with chills. ', 'DENGUE'), ("I've been having breathing issues, a persistent cough, and exhaustion. I've been coughing up a lot of thick, mucoid sputum and have a very h

In [57]:
from spacy.cli.train import train as spacy_train 

config_path = "config.cfg"
output_model_path = "./output/spacy_textcat"
spacy_train(
    config_path, 
    output_path=output_model_path,
    overrides={
        "paths.train":"train.spacy",
        "paths.dev":"dev.spacy"
    }
)

ℹ Saving to output directory: output\spacy_textcat
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.04        0.00    0.00
200     200          0.61        0.00    0.00
400     400          0.00        0.00    0.00
600     600          0.00        0.00    0.00


KeyboardInterrupt: 

In [58]:
!python -m spacy train config.cfg --paths.train train.spacy --paths.dev dev.spacy --output model --verbose

[i] Saving to output directory: model
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['textcat']
[i] Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.04        0.00    0.00
200     200          0.61        0.00    0.00
400     400          0.00        0.00    0.00
600     600          0.00        0.00    0.00
800     800          0.00        0.00    0.00
1000    1000          0.00        0.00    0.00
1200    1200          0.00        0.00    0.00
1400    1400          0.00        0.00    0.00
1600    1600          0.00        0.00    0.00
[+] Saved pipeline to output directory
model\model-last


[2023-05-02 19:54:49,551] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2023-05-02 19:54:49,842] [INFO] Set up nlp object from config
[2023-05-02 19:54:49,847] [DEBUG] Loading corpus from path: dev.spacy
[2023-05-02 19:54:49,848] [DEBUG] Loading corpus from path: train.spacy
[2023-05-02 19:54:49,848] [INFO] Pipeline: ['textcat']
[2023-05-02 19:54:49,849] [INFO] Created vocabulary
[2023-05-02 19:54:49,850] [INFO] Finished initializing nlp object
[2023-05-02 19:54:49,902] [INFO] Initialized pipeline components: ['textcat']
[2023-05-02 19:54:49,908] [DEBUG] Loading corpus from path: dev.spacy
[2023-05-02 19:54:49,909] [DEBUG] Loading corpus from path: train.spacy
[2023-05-02 19:54:49,913] [DEBUG] Removed existing output directory: model\model-best
[2023-05-02 19:54:49,916] [DEBUG] Removed existing output directory: model\model-last


In [26]:
import spacy 
trained_nlp = spacy.load("./output/spacy_textcat/model-best")
 
text = "I have a headache"
doc = trained_nlp(text)
doc.cats

{'PSORIASIS': 0.04082758352160454,
 'VARICOSE VEINS': 0.04082758352160454,
 'TYPHOID': 0.04082758352160454,
 'CHICKEN POX': 0.04082758352160454,
 'IMPETIGO': 0.04082758352160454,
 'DENGUE': 0.04082758352160454,
 'FUNGAL INFECTION': 0.04082758352160454,
 'COMMON COLD': 0.04082758352160454,
 'PNEUMONIA': 0.04082758352160454,
 'DIMORPHIC HEMORRHOIDS': 0.06096552684903145,
 'ARTHRITIS': 0.04082758352160454,
 'ACNE': 0.04082758352160454,
 'BRONCHIAL ASTHMA': 0.04082758352160454,
 'HYPERTENSION': 0.04082758352160454,
 'MIGRAINE': 0.04082758352160454,
 'CERVICAL SPONDYLOSIS': 0.04082758352160454,
 'JAUNDICE': 0.04082758352160454,
 'MALARIA': 0.04082758352160454,
 'URINARY TRACT INFECTION': 0.04082758352160454,
 'ALLERGY': 0.04082758352160454,
 'GASTROESOPHAGEAL REFLUX DISEASE': 0.04082758352160454,
 'DRUG REACTION': 0.04082758352160454,
 'PEPTIC ULCER DISEASE': 0.04082758352160454,
 'DIABETES': 0.04082758352160454}

In [56]:
!python -m spacy init config --lang en --pipeline textcat --optimize efficiency --force config.cfg

[i] Generated config template specific for your use case
- Language: en
- Pipeline: textcat
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
